# Latauslaskuri

Tällä laskurilla voit laskea, kuinka paljon sähköauton (tai lataushybridin)
lataus maksaa pörssisähköä käyttäen.
Kulutusdata annetaan minuuttikohtaisesti CSV-muodossa, ja pörssisähkön
hintatiedot ladataan automaattisesti latausjaksolle.
Laskussa huomioidaan pörssisähkön hinnan lisäksi kiinteät kustannukset eli
pörssisähkösopimuksen mukainen marginaali, sähkönsiirto sekä sähkövero.

In [13]:
from datetime import datetime
import os

import numpy as np
import pandas as pd
import requests

Määritetään sähkön kiinteät kustannukset senteissä per kilowattitunti
(snt kWh⁻¹).

In [14]:
MARGINAALI = 0.49
SIIRTO = 3.0
SAHKOVERO = 2.82752

kiinteat = MARGINAALI + SIIRTO + SAHKOVERO

### Kulutuksen lukeminen CSV-tiedostoista

Alla olevassa solussa kulutustiedot luetaan datakansiossa olevista
CSV-tiedostoista pandas DataFrameen.
Tämän jälkeen minuuttikohtainen kulutusdata muutetaan varttikohtaiseksi
summaamalla, ja datan yksiköt muunnetaan wattitunneista (Wh) kilowattitunteihin
(kWh) jakamalla varttikohtainen kulutuslukeman tuhannella.
Datasta poistetaan latausaseman valmiustilan kulutus asettamalla kaikki alle
1 Wh:n kulutuslukeman nollaan.

Shellyltä saatava CSV-tiedosto sisältää kulutusdatan lisäksi myös tiedot
sähköntuotannosta.
Nämä jätetään lukematta asettamalla `max_rows`-parametrille arvoksi 60.

In [15]:
minuuttikulutus = pd.DataFrame()

data_dir = './data'
directory = os.fsencode(data_dir)

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith('.csv'):
        ajat, kulutukset = np.loadtxt(
            os.path.join(data_dir, filename), dtype='str',
            delimiter=',', skiprows=2, unpack=True, max_rows=60)
        
        ajat = np.array(
            [datetime.strptime(i.strip(), '%d/%m/%Y %H:%M') for i in ajat])
        kulutukset = np.array(
            [float(i) for i in kulutukset])
        minuuttikulutus = pd.concat(
            [minuuttikulutus, pd.DataFrame(kulutukset, index=ajat)])

minuuttikulutus.columns = ['kulutus (kWh)']

minuuttikulutus.loc[minuuttikulutus.loc[:, 'kulutus (kWh)'] < 1] = 0

varttikulutus = minuuttikulutus.resample(
    pd.Timedelta(minutes=15)).sum().div(1e3)

### Latauksen hinnan laskeminen

Seuraavaksi luodaan uusi pandas DataFrame, jossa indeksinä on varttikohtainen
aika, ja sarakkeina pörssisähkön hinta (yksiköissä snt kWh⁻¹), sähkönkulutus
(yksiköissä kWh), ja kulutuskohtainen hinta (yksiköissä snt).

In [16]:
latauksen_hinta = pd.DataFrame(
    index=varttikulutus.index,
    columns=['pörssi (snt/kWh)', 'kulutus (kWh)', 'hinta (snt)'])

Tämän jälkeen ladataan pörssisähkön varttikohtaiset hinnat tarkastellulle
aikavälille.
Tässä tulee huomioida, että kulutusdatan aikavyöhyke on Suomen aikavyöhyke eli
UTC+2 talvi- ja UTC+3 kesäajalle, kun taas pörssisähkön hintatiedot annetaan
UTC-ajassa.
Tämän takia `latauksen_hinta` DataFramen indeksistä otettavat ajat on hintoja
ladattaessa muutettava UTC-aikaan, jotta oikea pörssisähkön hinta kohdistuu
oikealle kulutuslukemalle.

HUOM!
Lisää kesä-/talviajan käsittely, tämä tapahtuu tällä hetkellä manuaalisesti.

In [17]:
for aika in latauksen_hinta.index:
    aika_utc = aika - pd.Timedelta(hours=3)
    aika_str = aika_utc.strftime('%Y-%m-%dT%H:%M:%S.%fZ')
    hinta = requests.get(
        'https://api.porssisahko.net/v2/price.json?date=' + aika_str).json()
    latauksen_hinta.loc[aika, 'pörssi (snt/kWh)'] = hinta['price']

Lopuksi lasketaan auton latauksen varttikohtainen hinta lisäämällä pörssisähkön
hintoihin kiinteät kulut ja kertomalla kulutuslukemat näillä summilla.
Latauksen kokonaishinta saadaan sitten summaamalla varttikohtaiset hinnat
yhteen.
Varttikohtaiset hinnat ovat senteissä, joten lopputulos muutetaan vielä euroiksi
jakamalla lopullinen summa sadalla.

In [18]:
latauksen_hinta.loc[varttikulutus.index, 'kulutus (kWh)'] = (
    varttikulutus.loc[:, 'kulutus (kWh)'])
latauksen_hinta.loc[:, 'hinta (snt)'] = (
    (latauksen_hinta.loc[:, 'pörssi (snt/kWh)'] + kiinteat)
    *latauksen_hinta.loc[:, 'kulutus (kWh)'])

print(
    f'Auton lataus maksoi kokonaisuudessaan',
    f'{latauksen_hinta.loc[:, 'hinta (snt)'].div(1e2).sum():.2f}€.')

Auton lataus maksoi kokonaisuudessaan 0.70€.
